In [72]:
example1 = """#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#."""

example2 = """#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#"""

full_example_str = """#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#"""

import pandas as pd

def convert_matrix_str_to_df(matrix_str):
    return pd.DataFrame.from_records([list(x) for x in matrix_str.split('\n')])

example_df1 = convert_matrix_str_to_df(example1)
example_df2 = convert_matrix_str_to_df(example2)
example_df1

,0,1,2,3,4,5,6,7,8
0,#,.,#,#,.,.,#,#,.
1,.,.,#,.,#,#,.,#,.
2,#,#,.,.,.,.,.,.,#
3,#,#,.,.,.,.,.,.,#
4,.,.,#,.,#,#,.,#,.
5,.,.,#,#,.,.,#,#,.
6,#,.,#,.,#,#,.,#,.


In [51]:
# group rows into equivalence classes
from collections import defaultdict

row_classes = defaultdict(list)
for i, row in enumerate(example_df1.iterrows()):
    row_tuple = tuple(row[1].to_list())
    row_classes[row_tuple].append(i)
    
row_classes

defaultdict(list,
            {('#', '.', '#', '#', '.', '.', '#', '#', '.'): [0],
             ('.', '.', '#', '.', '#', '#', '.', '#', '.'): [1, 4],
             ('#', '#', '.', '.', '.', '.', '.', '.', '#'): [2, 3],
             ('.', '.', '#', '#', '.', '.', '#', '#', '.'): [5],
             ('#', '.', '#', '.', '#', '#', '.', '#', '.'): [6]})

In [52]:
mult_classes = [x for x in row_classes.values() if len(x) > 1]
mult_classes

[[1, 4], [2, 3]]

In [53]:
# find the equivalence class with interval 1; then work outward and see if the expected equivalence
# classes exist until we reach the end of the dataframe either up or down
breaking_rows = None
for mult_class in mult_classes:
    if abs(mult_class[0] - mult_class[1]) == 1:
        breaking_rows = mult_class
        break

row1, row2 = min(breaking_rows), max(breaking_rows)
reflect = True
while row1 >= 0 and row2 < example_df1.shape[0]:
    if [row1, row2] not in mult_classes:
        reflect = False
        break
    row1 += 1
    row2 -= 1
    
reflect


False

In [101]:
def compute_row_equivalence_classes(df):
    row_classes = defaultdict(list)
    for i, row in enumerate(df.iterrows()):
        row_tuple = tuple(row[1].to_list())
        row_classes[row_tuple].append(i)
    return row_classes


def compute_mirror_line_if_exists(mult_classes, df_boundary):
    # find the equivalence class with interval 1; then work outward and see if the expected equivalence
    # classes exist until we reach the end of the dataframe either up or down (or left/right if cols)
    rows_1_apart = []
    for mult_class in mult_classes:
        if abs(mult_class[0] - mult_class[1]) == 1:
            rows_1_apart.append(mult_class)
            
    if not rows_1_apart:  # no equivalence classes with interval 1
        return -1
            
    for mult_class in rows_1_apart:
        row1, row2 = min(mult_class), max(mult_class)
        breaking_row = row1
        
        reflect = True
        while row1 >= 0 and row2 <= df_boundary-1:
            if [row1, row2] not in mult_classes:
                reflect = False
                break
            row1 -= 1
            row2 += 1
        
        # if we've found the mirror, return it
        if reflect:
            return breaking_row

    # if we're here, none of the 1-apart rows were on a mirror line
    return -1

"""
Returns the index of the row right above the reflecting line, or -1 if doesn't exist
"""
def reflecting_row_num(df):
    row_classes = compute_row_equivalence_classes(df)
    mult_classes = [x for x in row_classes.values() if len(x) > 1]
    return compute_mirror_line_if_exists(mult_classes, df.shape[0])

In [93]:
print(reflecting_row_num(example_df1))
print(reflecting_row_num(example_df2))

-1
3


In [102]:
def compute_col_equivalence_classes(df):
    col_classes = defaultdict(list)
    for i in range(df.shape[1]):
        col_tuple = tuple(df[i].to_list())
        col_classes[col_tuple].append(i)

    return col_classes

"""
Returns the name of the col to the left of the reflecting line, or -1 if doesn't exist
"""
def reflecting_col_num(df):
    col_classes = compute_col_equivalence_classes(df)
    mult_classes = [x for x in col_classes.values() if len(x) > 1]
    return compute_mirror_line_if_exists(mult_classes, df.shape[1])

print(reflecting_col_num(example_df1))
print(reflecting_col_num(example_df2))

4
-1


In [103]:
def part1(matrix_strs):
    total = 0
    for example in matrix_strs.split('\n\n'):
        df = convert_matrix_str_to_df(example)
        col = reflecting_col_num(df)
        row = reflecting_row_num(df)
        if row > -1:
            total += 100*(row+1)
        if col > -1:
            total += col+1

    return total

part1(full_example_str)

405

In [105]:
a = open('day_13.txt').read()
#part1(a)

for example in a.split('\n\n'):
    print(example)
    df = convert_matrix_str_to_df(example)
    col = reflecting_col_num(df)
    row = reflecting_row_num(df)
    if row > -1:
        print('Row: %d\n' % (row+1))
    if col > -1:
        print('Col: %d\n' % (col+1))
    if row + col == -2:
        print('Nada\n')

.#..#......
..#.#......
..#...#....
#.##...####
.#..#..####
#.#.##.####
###..#.#..#
Col: 9

.#.##.#.###
..####..##.
#########..
.##..##..##
.##..##..##
#########..
..####..##.
.#.##.#.###
#.#..#.##.#
..#####.###
...##.....#
..#..#..#..
...##...#..
Row: 4

..##.#.
#...###
#...###
..##.##
..#..#.
....##.
#.#.#..
.#...##
##.#...
###.###
###.###
##.#...
.#...##
Row: 10

#.#.###
..##.##
...####
##....#
##....#
...####
..##.##
#.#.###
#.#####
....##.
.#...#.
#.#####
#....##
#..##.#
#..##.#
#..#.##
#.#####
Row: 4

###...#...#.#..
.#.##.#.....#..
..###..#..#....
..###..#..#....
.#.##.#........
###...#...#.#..
###..#..##.#.##
Nada

##.#.....
##.#...#.
..#.#..##
..##....#
#..#....#
#.#..##..
#.#..##..
#..#....#
..##....#
..#.#..##
##.#...#.
##.#.....
#.#..#.#.
Row: 6

#..#..#..#.##
..........###
..........###
#..#..#..#.##
###....####.#
....##......#
##..##..##..#
###....####..
....##.....##
.#......#.#.#
.##....##..##
..#.##.#....#
####..######.
.##..#.##.###
..##..##...#.
Row: 2

..#.###..#.
#


...#.##..
..#.####.
####.##.#
###.#..#.
##.#.##.#
..#..##..
###......
....#..#.
.....##..
####....#
...#.##.#
..#..##..
##.######
Col: 1

#.##.....
..#.#..#.
..#.#..#.
#.##.....
.##.....#
##.##..##
..###.###
.##..##..
.##..##..
..###.###
##.##..##
.##.....#
#.###....
Row: 2

#....#.##.####...
##.#...#..#..#.#.
####..####.######
..#...#...#...##.
###....#.#.#####.
###.#..##.....#.#
##.####.##.##..##
....##...##..#..#
##.....#.#.#...##
###.#.##.#.##..##
..##..##....#.###
##.##..#...###...
...###.#...####.#
...#....##.....##
##########...#..#
##....#.#.#......
##....#.#.#......
Row: 16

###.##.####
.##.##.##.#
#####.####.
#.#.##.#.#.
.#..##..#.#
..##..##...
#...##...##
#.#.##.#.##
##.####.##.
####..#####
..##..##..#
.##....##.#
.##....##.#
..##..##..#
####..#####
##.####.##.
#.#.##.#.##
Row: 12

.###............#
..#..##......##..
###....#....##...
###..####..####..
.....##.#..#.##..
..###.###..###.##
.##....#.##.#....
..#...#......#...
#..#.###.##.###.#
#.##...#.##.#...#
..#.##.#.##.#.#

In [110]:
example_str = """###...#...#.#..
.#.##.#.....#..
..###..#..#....
..###..#..#....
.#.##.#........
###...#...#.#..
###..#..##.#.##"""

df = convert_matrix_str_to_df(example_str)

col_classes = compute_col_equivalence_classes(df)
col_classes
# row1, row2 = None, None
# for mult_class in mult_classes:
#     if abs(mult_class[0] - mult_class[1]) == 1:
#         row1, row2 = min(mult_class), max(mult_class)
#         breaking_row = row1
        
# print(breaking_row)

defaultdict(list,
            {('#', '.', '.', '.', '.', '#', '#'): [0],
             ('#', '#', '.', '.', '#', '#', '#'): [1],
             ('#', '.', '#', '#', '.', '#', '#'): [2],
             ('.', '#', '#', '#', '#', '.', '.'): [3, 4],
             ('.', '.', '.', '.', '.', '.', '#'): [5, 8, 9, 11, 13, 14],
             ('#', '#', '.', '.', '#', '#', '.'): [6],
             ('.', '.', '#', '#', '.', '.', '.'): [7],
             ('#', '.', '#', '#', '.', '#', '.'): [10],
             ('#', '#', '.', '.', '.', '#', '.'): [12]})

2
